In [2]:
from __future__ import print_function, division
from builtins import range
from future.utils import iteritems
from nltk.corpus import brown
import operator
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('..'))
import matplotlib.pyplot as plt
import random
from datetime import datetime
from nltk import pos_tag, word_tokenize
import os
import sys
from tqdm import tqdm

In [3]:
def get_bigram_probs(sentences, V, start_idx, end_idx, smoothing=1):
  # structure of bigram probability matrix will be:
  # (last word, current word) --> probability
  # we will use add-1 smoothing
  # note: we'll always ignore this from the END token
  bigram_probs = np.ones((V, V)) * smoothing
  for sentence in sentences:
    for i in range(len(sentence)):
      
      if i == 0:
        # beginning word
        bigram_probs[start_idx, sentence[i]] += 1
      else:
        # middle word
        bigram_probs[sentence[i-1], sentence[i]] += 1

      # if we're at the final word
      # we update the bigram for last -> current
      # AND current -> END token
      if i == len(sentence) - 1:
        # final word
        bigram_probs[sentence[i], end_idx] += 1

  # normalize the counts along the rows to get probabilities
  bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
  return bigram_probs



In [4]:
KEEP_WORDS = set([
  'king', 'man', 'queen', 'woman',
  'italy', 'rome', 'france', 'paris',
  'london', 'britain', 'england',
])


def get_sentences():
    return brown.sents()

In [5]:
def get_sentences_with_word2idx():
  sentences = get_sentences()
  indexed_sentences = []

  i = 2
  word2idx = {'START': 0, 'END': 1}
  for sentence in sentences:
    indexed_sentence = []
    for token in sentence:
      token = token.lower()
      if token not in word2idx:
        word2idx[token] = i
        i += 1

      indexed_sentence.append(word2idx[token])
    indexed_sentences.append(indexed_sentence)

  print("Vocab size:", i)
  return indexed_sentences, word2idx

In [6]:
def get_sentences_with_word2idx_limit_vocab(n_vocab=2000, keep_words=KEEP_WORDS):
  sentences = get_sentences()
  indexed_sentences = []

  i = 2
  word2idx = {'START': 0, 'END': 1}
  idx2word = ['START', 'END']

  word_idx_count = {
    0: float('inf'),
    1: float('inf'),
  }

  for sentence in sentences:
    indexed_sentence = []
    for token in sentence:
      token = token.lower()
      if token not in word2idx:
        idx2word.append(token)
        word2idx[token] = i
        i += 1

      # keep track of counts for later sorting
      idx = word2idx[token]
      word_idx_count[idx] = word_idx_count.get(idx, 0) + 1

      indexed_sentence.append(idx)
    indexed_sentences.append(indexed_sentence)



  # restrict vocab size

  # set all the words I want to keep to infinity
  # so that they are included when I pick the most
  # common words
  for word in keep_words:
    word_idx_count[word2idx[word]] = float('inf')

  sorted_word_idx_count = sorted(word_idx_count.items(), key=operator.itemgetter(1), reverse=True)
  word2idx_small = {}
  new_idx = 0
  idx_new_idx_map = {}
  for idx, count in sorted_word_idx_count[:n_vocab]:
    word = idx2word[idx]
    print(word, count)
    word2idx_small[word] = new_idx
    idx_new_idx_map[idx] = new_idx
    new_idx += 1
  # let 'unknown' be the last token
  word2idx_small['UNKNOWN'] = new_idx 
  unknown = new_idx

  assert('START' in word2idx_small)
  assert('END' in word2idx_small)
  for word in keep_words:
    assert(word in word2idx_small)

  # map old idx to new idx
  sentences_small = []
  for sentence in indexed_sentences:
    if len(sentence) > 1:
      new_sentence = [idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence]
      sentences_small.append(new_sentence)

  return sentences_small, word2idx_small

In [7]:



def init_weight(Mi, Mo):
    return np.random.randn(Mi, Mo) / np.sqrt(Mi + Mo)

def all_parity_pairs(nbit):
    # total number of samples (Ntotal) will be a multiple of 100
    # why did I make it this way? I don't remember.
    N = 2**nbit
    remainder = 100 - (N % 100)
    Ntotal = N + remainder
    X = np.zeros((Ntotal, nbit))
    Y = np.zeros(Ntotal)
    for ii in range(Ntotal):
        i = ii % N
        # now generate the ith sample
        for j in range(nbit):
            if i % (2**(j+1)) != 0:
                i -= 2**j
                X[ii,j] = 1
        Y[ii] = X[ii].sum() % 2
    return X, Y

def all_parity_pairs_with_sequence_labels(nbit):
    X, Y = all_parity_pairs(nbit)
    N, t = X.shape

    # we want every time step to have a label
    Y_t = np.zeros(X.shape, dtype=np.int32)
    for n in range(N):
        ones_count = 0
        for i in range(t):
            if X[n,i] == 1:
                ones_count += 1
            if ones_count % 2 == 1:
                Y_t[n,i] = 1

    X = X.reshape(N, t, 1).astype(np.float32)
    return X, Y_t

# unfortunately Python 2 and 3 translates work differently
def remove_punctuation_2(s):
    return s.translate(None, string.punctuation)

def remove_punctuation_3(s):
    return s.translate(str.maketrans('','',string.punctuation))

if sys.version.startswith('2'):
    remove_punctuation = remove_punctuation_2
else:
    remove_punctuation = remove_punctuation_3


def get_robert_frost():
    word2idx = {'START': 0, 'END': 1}
    current_idx = 2
    sentences = []
    for line in open('../hmm_class/robert_frost.txt'):
        line = line.strip()
        if line:
            tokens = remove_punctuation(line.lower()).split()
            sentence = []
            for t in tokens:
                if t not in word2idx:
                    word2idx[t] = current_idx
                    current_idx += 1
                idx = word2idx[t]
                sentence.append(idx)
            sentences.append(sentence)
    return sentences, word2idx

def my_tokenizer(s):
    s = remove_punctuation(s)
    s = s.lower() # downcase
    return s.split()

def get_wikipedia_data(n_files, n_vocab, by_paragraph=False):
    prefix = '/'

    if not os.path.exists(prefix):
        print("Are you sure you've downloaded, converted, and placed the Wikipedia data into the proper folder?")
        print("I'm looking for a folder called large_files, adjacent to the class folder, but it does not exist.")
        print("Please download the data from https://dumps.wikimedia.org/")
        print("Quitting...")
#        exit()

    input_files = [f for f in os.listdir(prefix) if f.startswith('enwiki') and f.endswith('txt')]

    if len(input_files) == 0:
        print("Looks like you don't have any data files, or they're in the wrong location.")
        print("Please download the data from https://dumps.wikimedia.org/")
        print("Quitting...")
#        exit()

    # return variables
    sentences = []
    word2idx = {'START': 0, 'END': 1}
    idx2word = ['START', 'END']
    current_idx = 2
    word_idx_count = {0: float('inf'), 1: float('inf')}

    if n_files is not None:
        input_files = input_files[:n_files]

    for f in input_files:
        print("reading:", f)
        for line in open(prefix + f):
            line = line.strip()
            # don't count headers, structured data, lists, etc...
            if line and line[0] not in ('[', '*', '-', '|', '=', '{', '}'):
                if by_paragraph:
                    sentence_lines = [line]
                else:
                    sentence_lines = line.split('. ')
                for sentence in sentence_lines:
                    tokens = my_tokenizer(sentence)
                    for t in tokens:
                        if t not in word2idx:
                            word2idx[t] = current_idx
                            idx2word.append(t)
                            current_idx += 1
                        idx = word2idx[t]
                        word_idx_count[idx] = word_idx_count.get(idx, 0) + 1
                    sentence_by_idx = [word2idx[t] for t in tokens]
                    sentences.append(sentence_by_idx)

    # restrict vocab size
    sorted_word_idx_count = sorted(word_idx_count.items(), key=operator.itemgetter(1), reverse=True)
    word2idx_small = {}
    new_idx = 0
    idx_new_idx_map = {}
    for idx, count in sorted_word_idx_count[:n_vocab]:
        word = idx2word[idx]
        print(word, count)
        word2idx_small[word] = new_idx
        idx_new_idx_map[idx] = new_idx
        new_idx += 1
    # let 'unknown' be the last token
    word2idx_small['UNKNOWN'] = new_idx 
    unknown = new_idx

    assert('START' in word2idx_small)
    assert('END' in word2idx_small)
    assert('king' in word2idx_small)
    assert('queen' in word2idx_small)
    assert('man' in word2idx_small)
    assert('woman' in word2idx_small)

    # map old idx to new idx
    sentences_small = []
    for sentence in sentences:
        if len(sentence) > 1:
            new_sentence = [idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence]
            sentences_small.append(new_sentence)

    return sentences_small, word2idx_small

def get_tags(s):
    tuples = pos_tag(word_tokenize(s))
    return [y for x, y in tuples]

def get_poetry_classifier_data(samples_per_class, load_cached=True, save_cached=True):
    datafile = 'poetry_classifier_data.npz'
    if load_cached and os.path.exists(datafile):
        npz = np.load(datafile)
        X = npz['arr_0']
        Y = npz['arr_1']
        V = int(npz['arr_2'])
        return X, Y, V

    word2idx = {}
    current_idx = 0
    X = []
    Y = []
    for fn, label in zip(('../hmm_class/edgar_allan_poe.txt', '../hmm_class/robert_frost.txt'), (0, 1)):
        count = 0
        for line in open(fn):
            line = line.rstrip()
            if line:
                print(line)
                # tokens = remove_punctuation(line.lower()).split()
                tokens = get_tags(line)
                if len(tokens) > 1:
                    # scan doesn't work nice here, technically could fix...
                    for token in tokens:
                        if token not in word2idx:
                            word2idx[token] = current_idx
                            current_idx += 1
                    sequence = np.array([word2idx[w] for w in tokens])
                    X.append(sequence)
                    Y.append(label)
                    count += 1
                    print(count)
                    # quit early because the tokenizer is very slow
                    if count >= samples_per_class:
                        break
    if save_cached:
        np.savez(datafile, X, Y, current_idx)
    return X, Y, current_idx


def get_stock_data():
    input_files = os.listdir('stock_data')
    min_length = 2000

    # first find the latest start date
    # so that each time series can start at the same time
    max_min_date = datetime(2000, 1, 1)
    line_counts = {}
    for f in input_files:
        n = 0
        for line in open('stock_data/%s' % f):
            # pass
            n += 1
        line_counts[f] = n
        if n > min_length:
            # else we'll ignore this symbol, too little data
            # print 'stock_data/%s' % f, 'num lines:', n
            last_line = line
            date = line.split(',')[0]
            date = datetime.strptime(date, '%Y-%m-%d')
            if date > max_min_date:
                max_min_date = date

    print("max min date:", max_min_date)

    # now collect the data up to min date
    all_binary_targets = []
    all_prices = []
    for f in input_files:
        if line_counts[f] > min_length:
            prices = []
            binary_targets = []
            first = True
            last_price = 0
            for line in open('stock_data/%s' % f):
                if first:
                    first = False
                    continue
                date, price = line.split(',')[:2]
                date = datetime.strptime(date, '%Y-%m-%d')
                if date < max_min_date:
                    break
                prices.append(float(price))
                target = 1 if last_price < price else 0
                binary_targets.append(target)
                last_price = price
            all_prices.append(prices)
            all_binary_targets.append(binary_targets)

    # D = number of symbols
    # T = length of series
    return np.array(all_prices).T, np.array(all_binary_targets).T # make it T x D



In [11]:


sentences, word2idx = get_sentences_with_word2idx_limit_vocab(10000)
# sentences, word2idx = get_sentences_with_word2idx()

# vocab size
V = len(word2idx)
print("Vocab size:", V)

# we will also treat beginning of sentence and end of sentence as bigrams
# START -> first word
# last word -> END
start_idx = word2idx['START']
end_idx = word2idx['END']


# a matrix where:
# row = last word
# col = current word
# value at [row, col] = p(current word | last word)
bigram_probs = get_bigram_probs(sentences, V, start_idx, end_idx, smoothing=0.1)


# a function to calculate normalized log prob score
# for a sentence
def get_score(sentence):
    score = 0
    for i in range(len(sentence)):
      if i == 0:
        # beginning word
        score += np.log(bigram_probs[start_idx, sentence[i]])
      else:
        # middle word
        score += np.log(bigram_probs[sentence[i-1], sentence[i]])
    # final word
    score += np.log(bigram_probs[sentence[-1], end_idx])

    # normalize the score
    return score / (len(sentence) + 1)


# a function to map word indexes back to real words
idx2word = dict((v, k) for k, v in iteritems(word2idx))
def get_words(sentence):
    return ' '.join(idx2word[i] for i in sentence)


# when we sample a fake sentence, we want to ensure not to sample
# start token or end token
sample_probs = np.ones(V)
sample_probs[start_idx] = 0
sample_probs[end_idx] = 0
sample_probs /= sample_probs.sum()

# test our model on real and fake sentences
while True:
    # real sentence
    real_idx = np.random.choice(len(sentences))
    real = sentences[real_idx]

    # fake sentence
    fake = np.random.choice(V, size=len(real), p=sample_probs)

    print("REAL:", get_words(real), "SCORE:", get_score(real))
    print("FAKE:", get_words(fake), "SCORE:", get_score(fake))

    # input your own sentence
    custom = input("Enter your own sentence:\n")
    custom = custom.lower().split()

    # check that all tokens exist in word2idx (otherwise, we can't get score)
    bad_sentence = False
    for token in custom:
      if token not in word2idx:
        bad_sentence = True

    if bad_sentence:
      print("Sorry, you entered words that are not in the vocabulary")
    else:
      # convert sentence into list of indexes
      custom = [word2idx[token] for token in custom]
      print("SCORE:", get_score(custom))


    cont = input("Continue? [Y/n]")
    if cont and cont.lower() in ('N', 'n'):
      break



START inf
END inf
man inf
paris inf
britain inf
england inf
king inf
woman inf
rome inf
london inf
queen inf
italy inf
france inf
the 69971
, 58334
. 49346
of 36412
and 28853
to 26158
a 23195
in 21337
that 10594
is 10109
was 9815
he 9548
for 9489
`` 8837
'' 8789
it 8760
with 7289
as 7253
his 6996
on 6741
be 6377
; 5566
at 5372
by 5306
i 5164
this 5145
had 5133
? 4693
not 4610
are 4394
but 4381
from 4370
or 4206
have 3942
an 3740
they 3620
which 3561
-- 3432
one 3292
you 3286
were 3284
her 3036
all 3001
she 2860
there 2728
would 2714
their 2669
we 2652
him 2619
been 2472
) 2466
has 2437
( 2435
when 2331
who 2252
will 2245
more 2215
if 2198
no 2139
out 2097
so 1985
said 1961
what 1908
up 1890
its 1858
about 1815
: 1795
into 1791
than 1790
them 1788
can 1772
only 1748
other 1702
new 1635
some 1618
could 1601
time 1598
! 1596
these 1573
two 1412
may 1402
then 1380
do 1363
first 1361
any 1344
my 1318
now 1314
such 1303
like 1292
our 1252
over 1236
me 1181
even 1170
most 1159
made 1125
also 

image 119
decision 119
window 119
religion 119
aj 118
gun 118
responsibility 118
middle 118
europe 118
british 118
character 118
learned 117
horse 117
writing 117
appear 117
s. 117
account 117
ones 116
serious 116
activity 116
types 116
green 116
length 116
lived 115
audience 115
letters 115
returned 115
obtained 115
nuclear 115
specific 115
corner 115
forward 115
activities 115
slowly 115
doubt 114
6 114
justice 114
moving 114
latter 114
gives 114
straight 114
hit 114
plane 114
quality 114
design 114
obviously 114
operation 113
plans 113
shot 113
seven 113
a. 113
choice 113
poor 113
staff 113
function 113
figures 113
parts 113
stay 113
saying 113
include 113
15 113
born 113
pattern 113
30 112
cars 112
whatever 112
sun 112
faith 111
pool 111
hospital 110
corps 110
wish 110
lack 110
completely 110
heavy 110
waiting 110
speak 110
ball 110
standard 110
extent 110
visit 109
democratic 109
firm 109
income 109
ahead 109
deep 109
there's 109
language 109
principle 109
none 108
price 108
desig

officials 62
m. 62
senate 62
fund 62
9 62
billion 62
rising 62
11 62
speaking 62
internal 62
struggle 62
agencies 62
u. 62
december 62
equally 62
sets 62
please 62
drove 62
arrived 62
save 62
achieved 62
assignment 62
baby 62
guests 62
greatly 62
recognize 62
wilson 62
library 62
careful 62
pleasure 62
cool 62
extreme 62
concerning 62
governments 61
procedures 61
prices 61
duty 61
courses 61
friendly 61
we're 61
r. 61
coast 61
acting 61
50 61
closer 61
speech 61
european 61
showing 61
boston 61
victory 61
beach 61
minister 61
commercial 61
metal 61
possibly 61
tests 61
soft 61
kid 61
vast 61
continuing 61
associated 61
shoulder 61
weapons 61
shore 61
greek 61
travel 61
imagine 61
feelings 61
organizations 61
ideal 61
eat 61
friday 60
keeping 60
heavily 60
armed 60
ended 60
learning 60
text 60
existing 60
scale 60
setting 60
goal 60
task 60
contract 60
garden 60
nose 60
refused 60
streets 60
orchestra 60
contained 60
machinery 60
chemical 60
onto 60
circle 60
slow 60
maintain 60
fat 60


automatic 40
burned 40
permanent 40
observations 40
drawing 40
winston 40
desegregation 40
guidance 40
improvement 40
treasury 40
presumably 40
bars 40
brings 40
papa 40
indicates 40
discover 40
painted 40
intense 40
tool 40
necessity 40
eleven 40
shouted 40
focus 40
stepped 40
finger 40
conscience 40
criticism 40
thrown 40
glance 40
regions 40
stranger 40
joy 40
pope 40
e 40
atoms 40
visual 40
parallel 40
shear 40
rode 40
legislature 39
authorities 39
estimate 39
lawrence 39
acts 39
improve 39
rayburn 39
cooperation 39
communists 39
neutral 39
determination 39
deeply 39
assured 39
attractive 39
transfer 39
represents 39
colleges 39
joint 39
mississippi 39
severe 39
introduction 39
emergency 39
striking 39
trials 39
self 39
gained 39
contributed 39
mad 39
magazine 39
forever 39
mystery 39
tv 39
selection 39
code 39
anywhere 39
furniture 39
agents 39
derived 39
revealed 39
provisions 39
guest 39
allotment 39
satisfactory 39
controlled 39
finish 39
maturity 39
concert 39
comedy 39
stick 

sacrifice 30
et 30
network 30
johnny 30
eddie 30
dates 30
cuts 30
reveal 30
nowhere 30
comments 30
locked 30
ranging 30
controls 30
strip 30
alex 30
dealers 30
excessive 30
buying 30
associations 30
they'd 30
era 30
virtue 30
dreams 30
secure 30
sharpe 30
impressed 30
historian 30
listened 30
crucial 30
propaganda 30
deliberately 30
measuring 30
hoping 30
surprising 30
complicated 30
occurrence 30
preceding 30
skilled 30
radical 30
citizen 30
slave 30
altogether 30
purely 30
frontier 30
dimensions 30
root 30
blanket 30
encountered 30
consequence 30
consciousness 30
flux 30
shakespeare 30
cried 30
mixture 30
asleep 30
concentrated 30
meal 30
stable 30
stem 30
carbon 30
grinned 30
unconscious 30
dartmouth 30
sovereign 30
miriam 30
woodruff 30
fees 29
divorce 29
republicans 29
sherman 29
argued 29
tea 29
extending 29
utility 29
lieutenant 29
proposals 29
o. 29
questioned 29
modest 29
contributions 29
mighty 29
ignored 29
morse 29
allied 29
perform 29
transferred 29
false 29
guilty 29
meri

...

handicapped 13
familiarity 13
collapsed 13
omitted 13
barbecue 13
acid 13
scotland 13
birmingham 13
imply 13
nazi 13
explaining 13
abilities 13
sells 13
1900 13
sweeping 13
stimulation 13
undertake 13
lowest 13
inventories 13
assets 13
taxed 13
steep 13
ireland 13
truman 13
pepper 13
utter 13
lasting 13
shades 13
copper 13
carpet 13
couples 13
streetcar 13
walks 13
nonsense 13
understandable 13
performers 13
map 13
prayers 13
conducting 13
imperial 13
brutality 13
smallest 13
miserable 13
separately 13
rely 13
satisfying 13
owns 13
depreciation 13
derive 13
implicit 13
ideology 13
dictatorship 13
sentences 13
suspicious 13
rebellion 13
purple 13
electoral 13
rounds 13
roar 13
gifted 13
aloud 13
dumb 13
doubtless 13
pentagon 13
greene 13
collecting 13
journalism 13
investigators 13
ethnic 13
elite 13
consumed 13
planners 13
fiat 13
modernization 13
forcing 13
ton 13
biography 13
evaluate 13
vigorously 13
stability 13
creates 13
frankly 13
sector 13
specimens 13
prohibition 13
tempted 13

perfection 11
thirteen 11
ye 11
lengthy 11
blond 11
happier 11
hong 11
kong 11
drum 11
towels 11
proprietor 11
presbyterian 11
mortar 11
freeman 11
walnut 11
sculpture 11
entertained 11
easter 11
array 11
drastic 11
loading 11
necessitated 11
38 11
poorly 11
dodge 11
choosing 11
hardware 11
comprise 11
testified 11
dots 11
grimly 11
youths 11
pohl 11
subdivision 11
del 11
criteria 11
integrated 11
theaters 11
transfers 11
treating 11
claude 11
unusually 11
jimmy 11
hillsboro 11
showmanship 11
rabbit 11
poultry 11
mill 11
tide 11
ribs 11
drifting 11
alternate 11
incidents 11
wilderness 11
disarmament 11
sanitation 11
coincidence 11
1930 11
prompt 11
threaten 11
dillon 11
satisfactorily 11
discrepancy 11
climbing 11
paradoxically 11
acreage 11
planted 11
deductions 11
taxpayer 11
warn 11
entities 11
descent 11
prejudice 11
archaeology 11
buys 11
breast 11
juice 11
arched 11
wells 11
glaze 11
midwest 11
violin 11
here's 11
wired 11
appreciated 11
phillips 11
frantic 11
reunion 11
marking 

avocado 10
tasted 10
detect 10
employing 10
doomed 10
differs 10
non 10
wipe 10
launch 10
rating 10
screws 10
drilling 10
equations 10
harness 10
muzzle 10
boulder 10
roast 10
boiled 10
ernest 10
button 10
cigar 10
forge 10
graceful 10
mosque 10
thermometer 10
stains 10
coats 10
reinforce 10
container 10
keelson 10
mixing 10
ingenious 10
oxen 10
raid 10
paints 10
screens 10
insulation 10
negligible 10
vastly 10
transducer 10
gram 10
creep 10
defining 10
educator 10
deceased 10
tents 10
advancement 10
cellulose 10
three-dimensional 10
linguistic 10
sailed 10
anchored 10
weird 10
vue 10
imagery 10
psychologist 10
caring 10
fertility 10
concord 10
airplanes 10
messenger 10
doll 10
vagina 10
learns 10
accumulated 10
steichen 10
bacon 10
impatient 10
fake 10
jaws 10
brodie 10
utilized 10
duly 10
mice 10
refrain 10
comrades 10
legends 10
waiter 10
calculations 10
strode 10
scar 10
relevance 10
identities 10
instinctively 10
non-violent 10
fragments 10
swear 10
tunnel 10
spatial 10
segments 1

elect 8
feeble 8
tangle 8
retiring 8
corrupt 8
selfish 8
petty 8
rayburn's 8
slums 8
overly 8
succeeds 8
merchandising 8
inaugural 8
monuments 8
hoover 8
statues 8
gazing 8
guessing 8
predict 8
wreck 8
la. 8
revive 8
hemphill 8
contracted 8
investigating 8
managing 8
bankruptcy 8
travelers 8
repairs 8
$2 8
renewal 8
spends 8
bites 8
julian 8
years' 8
barred 8
mo. 8
teamsters 8
organizing 8
350 8
honoring 8
hugh 8
denomination 8
authoritative 8
miracles 8
super 8
church's 8
engaging 8
fraud 8
sullivan 8
blows 8
pitchers 8
barker 8
freshman 8
sheldon 8
colts 8
mating 8
rosy 8
knights 8
butcher 8
undergo 8
week's 8
stops 8
aerial 8
ankle 8
tech 8
exceptionally 8
comforting 8
leagues 8
charley 8
clicked 8
traveler 8
herb 8
slump 8
19th 8
cardinals 8
championship 8
babe 8
prominently 8
winner 8
charity 8
trophy 8
echoes 8
heed 8
gibson 8
flu 8
pitched 8
contented 8
peaks 8
resented 8
preached 8
inviting 8
ruth's 8
wendell 8
mccormick 8
niece 8
memphis 8
carnegie 8
generously 8
cheer 8
stag 

Enter your own sentence:
i love math
Sorry, you entered words that are not in the vocabulary
Continue? [Y/n]n


In [12]:
# load in the data
# note: sentences are already converted to sequences of word indexes
# note: you can limit the vocab size if you run out of memory
sentences, word2idx = get_sentences_with_word2idx_limit_vocab(2000)
# sentences, word2idx = get_sentences_with_word2idx()

KeyboardInterrupt: 

In [13]:
len(sentences) , len(word2idx)

(57013, 10001)

In [14]:
sentences[0]

[13,
 5434,
 661,
 2299,
 1639,
 74,
 1850,
 47,
 2180,
 16,
 10000,
 561,
 1129,
 1408,
 1202,
 26,
 71,
 483,
 27,
 21,
 97,
 9200,
 224,
 182,
 15]

In [15]:
for i,(a,b) in enumerate(word2idx.items()) : 
    if i ==20 : break
    print(a ,'     ', b)

START       0
END       1
man       2
paris       3
britain       4
england       5
king       6
woman       7
rome       8
london       9
queen       10
italy       11
france       12
the       13
,       14
.       15
of       16
and       17
to       18
a       19


In [16]:
word2idx['of']

16

In [17]:
# vocab size
V = len(word2idx)
print("Vocab size:", V)

Vocab size: 10001


In [18]:
# we will also treat beginning of sentence and end of sentence as bigrams
# START -> first word
# last word -> END
start_idx = word2idx['START']
end_idx = word2idx['END']
start_idx , end_idx

(0, 1)

In [19]:
# a matrix where:
# row = last word
# col = current word
# value at [row, col] = p(current word | last word)
bigram_probs = get_bigram_probs(sentences, V, start_idx, end_idx, smoothing=0.1)

In [20]:
bigram_probs.shape

(10001, 10001)

In [21]:
bigram_probs[0][:20]

array([1.72374860e-06, 1.72374860e-06, 2.60286039e-04, 1.89612346e-05,
       7.06736927e-05, 3.61987206e-05, 1.89612346e-05, 1.72374860e-06,
       1.72374860e-06, 5.34362066e-05, 1.89612346e-05, 1.72374860e-06,
       1.89612346e-05, 1.18164690e-01, 1.91336095e-04, 1.72374860e-06,
       2.70800905e-03, 1.95145579e-02, 7.39660525e-03, 1.95317954e-02])

In [22]:
# train a logistic model
W = np.random.randn(V, V) / np.sqrt(V)

In [23]:
losses = []
epochs = 1
lr = 1e-1

In [24]:
def softmax(a):
    a = a - a.max()
    exp_a = np.exp(a)
    return exp_a / exp_a.sum(axis=1, keepdims=True)

In [25]:
# what is the loss if we set W = log(bigram_probs)?
W_bigram = np.log(bigram_probs)
bigram_losses = []

In [26]:
W_bigram.shape

(10001, 10001)

In [27]:
W_bigram[0]

array([-13.27100922, -13.27100922,  -8.25372938, ..., -13.27100922,
       -13.27100922,  -3.14273946])

In [28]:
NN = 3000
t0 = datetime.now()
for epoch in range(epochs):
# shuffle sentences at each epoch
    random.shuffle(sentences)

    j = 0 # keep track of iterations
    for sentence in tqdm(sentences[:NN]):
        # convert sentence into one-hot encoded inputs and targets
        sentence = [start_idx] + sentence + [end_idx]
        n = len(sentence)
        inputs = np.zeros((n - 1, V))
        targets = np.zeros((n - 1, V))
        inputs[np.arange(n - 1), sentence[:n-1]] = 1
        targets[np.arange(n - 1), sentence[1:]] = 1

        # get output predictions
        predictions = softmax(inputs.dot(W))

        # do a gradient descent step
        W = W - lr * inputs.T.dot(predictions - targets)

        # keep track of the loss
        loss = -np.sum(targets * np.log(predictions)) / (n - 1)
        losses.append(loss)

        # keep track of the bigram loss
        # only do it for the first epoch to avoid redundancy
        if epoch == 0:
            bigram_predictions = softmax(inputs.dot(W_bigram))
            bigram_loss = -np.sum(targets * np.log(bigram_predictions)) / (n - 1)
            bigram_losses.append(bigram_loss)


#         if j % 10 == 0:
#             print("epoch:", epoch, "sentence: %s/%s" % (j, len(sentences[:NN])), "loss:", loss)
        j += 1

  0%|▏                                                                              | 6/3000 [00:15<2:09:54,  2.60s/it]

KeyboardInterrupt: 

In [ ]:
print("Elapsed time training:", datetime.now() - t0)
plt.plot(losses)

In [ ]:
# plot a horizontal line for the bigram loss
avg_bigram_loss = np.mean(bigram_losses)
print("avg_bigram_loss:", avg_bigram_loss)
plt.axhline(y=avg_bigram_loss, color='r', linestyle='-')

In [ ]:
# plot smoothed losses to reduce variability
def smoothed_loss(x, decay=0.99):
    
    y = np.zeros(len(x))
    last = 0
    for t in range(len(x)):
        z = decay * last + (1 - decay) * x[t]
        y[t] = z / (1 - decay ** (t + 1))
        last = z
    return y

In [ ]:
plt.plot(smoothed_loss(losses))
plt.show()